## 动态规划

### 1. 斐波那契数列

#### 递归解法

In [2]:
def fib(n):
    if n < 2:
        return n
    else:
        return fib(n-1) + fib(n-2)

In [3]:
fib(10)

55

In [19]:
import time

def comTime(func,n):
    start = time.time()
    func(n)
    end = time.time()
    return end-start

In [21]:
comTime(fib,30)

0.1972668170928955

时间复杂度**O(2^n)**,中间存在大量的**重叠子问题**。

#### 带备忘录的递归解法

In [6]:
# 错误写法，注意带备忘录的方法中存储中间态的数据结构的位置。
def fib2(n):
    d = {}
    if n < 2:
        return n
    d[0] = 0; d[1] = 1
    if n in d.keys():
        return d[n]
    else:
        temp = fib2(n-1) + fib2(n-2)
        d[n] = temp
        return temp

In [7]:
fib2(10)

55

In [22]:
comTime(fib2,30)

0.3563272953033447

从运行时间可见，函数fib2( )并没有加快速度，原因在于存储中间态的字典定义位置错误，应调整为嵌套函数形式。  
修正后1如fib3( )所示。

In [12]:
def digui(n,d):
    if n < 2:
        return n

    if n in d.keys():
        return d[n]
    else:
        temp = digui(n-1,d) + digui(n-2,d)
        d[n] = temp
        return temp
    
def fib3(n):
    d = {}; d[0] = 0; d[1] = 1
    res = digui(n,d)
    return res

In [13]:
fib3(10)

55

In [25]:
comTime(fib3,1000)

0.002212047576904297

时间复杂度**O(n)**。

#### 动态规划解法

In [16]:
def dp(n):
    res = [''] * (n+1)
    res[0] = 0; res[1] = 1
    
    for i in range(2,n+1):
        res[i] = res[i-1] + res[i-2]
        
    return res[n]

In [17]:
dp(10)

55

#### 改进的动态规划解法

In [18]:
def dp2(n):
    if n < 2:
        return n
    fir = 0; sec = 1
    
    for i in range(2,n+1):
        cur = fir + sec
        fir = sec
        sec = cur
        
    return cur

In [20]:
dp2(10)

55

递归是**自顶向下**结构，动态规划是**自底向上**。  
动态规划三要素为：**重叠子问题、最优子结构、状态转移方程**。  
严格来说，斐波那契数列不算动态规划，因为没有**最优子结构**。

### 2. 凑零钱问题

#### 暴力解法（递归）

In [26]:
def change(coins,amout):
    if amout == 0:
        return 0

    res = 1000
    for coin in coins:
        if amout-coin >= 0:
            res = min(res,1+change(coins,amout-coin))
        else:
            continue # 思考：continue处发生了什么？
    return res

In [32]:
change([1,2,5],11)

3

In [34]:
def comTime2(func,coins,amout):
    start = time.time()
    func(coins,amout)
    end = time.time()
    return end-start

In [42]:
comTime2(change,[1,2,5],31)

4.344850063323975

In [28]:
# 书中示例
def coinChange(coins,amout):
    def dp(n):
        if n == 0:
            return 0
        if n < 0:
            return -1
        res = 1000
        for coin in coins:
            subproblem = coinChange(coins,n-coin)
            if subproblem == -1: continue
            res = min(res,1+subproblem)
        return res
    return dp(amout)

In [33]:
coinChange([1,2,5],11)

3

In [43]:
comTime2(coinChange,[1,2,5],31)

9.435334920883179

上述写法都存在问题，无法处理**不能刚好找零**的情况。

In [59]:
change([2,5],3), coinChange([2,5],3)

(1000, 1000)

#### 带备忘录的递归

In [44]:
def change2(coins,amout):
    d = {}
    if amout == 0:
        return 0
    if amout in d.keys():
        return d[amout]

    res = 1000
    for coin in coins:
        if amout-coin >= 0:
            res = min(res,1+change2(coins,amout-coin))
        else:
            continue # 思考：continue处发生了什么？
    d[amout] = res
    return res

In [45]:
comTime2(change2,[1,2,5],31)

5.427884101867676

出现了**斐波那契数列问题**中相同的错误。修正如下：

In [48]:
def digui2(coins,amout,d):
    if amout == 0:
        return 0
    if amout in d.keys():
        return d[amout]

    res = 1000
    for coin in coins:
        if amout-coin >= 0:
            res = min(res,1+digui2(coins,amout-coin,d))
        else:
            continue
    d[amout] = res
    return res

def change3(coins,amout):
    d = {}
    return digui2(coins,amout,d)

In [50]:
comTime2(change3,[1,2,5],1112)

0.0033941268920898438

换一种写法，能否写成**一个函数形式**？

In [60]:
def change4(coins,amout,d):
    if amout == 0:
        return 0
    if amout in d.keys():
        return d[amout]

    res = 1000
    for coin in coins:
        if amout-coin >= 0:
            res = min(res,1+change4(coins,amout-coin,d))
        else:
            continue
    d[amout] = res
    return res

计算时间函数是否可写成通用形式？

In [62]:
# 用可变参数
def comTimeG(func,*n):
    import time
    start = time.time()
    func(*n)
    end = time.time()
    return end-start

In [63]:
comTimeG(fib2,30), comTimeG(change,[1,2,5],31)

(0.3535652160644531, 4.296720266342163)

In [64]:
comTimeG(change4,[1,2,5],1112,{})

0.004559755325317383

#### 动态规划解法

In [69]:
def dpChange(coins,amout):
    if amout in coins:
        return 1
    l = [0] * (amout + 1)
    for i in range(1,amout+1):
        res = i
        for coin in coins:
            if i - coin < 0:
                continue
            else:
                res = min(res,1 + l[i-coin])
        l[i] = res
    return l[amout]

In [71]:
dpChange([1,2,5],11)

3

书中：计算机解决问题，唯一的途径就是**穷举**，穷举所有可能性。  
算法设计⽆⾮就是先思考“如何穷举”，然后再追求“如何聪明地穷举”。

思考：可以**通过数学推理来优化算法**，从而“聪明地穷举”，忽略一些不可能的情况。

### 拓展

#### 什么是最优子结构？

「最优⼦结构」是某些问题的⼀种特定性质，并不是动态规划问题专有的。很多问题其实都具有最优⼦结构，只是其中⼤部分不具有重叠⼦问题，所以我们不把它们归为动态规划系列问题。  
**最优⼦结构：可以从⼦问题的最优结果推出更⼤规模问题的最优结果。**  
想满⾜最优⼦结构，**⼦问题之间必须互相独⽴**。

### 3. 最长递增子序列

给定一个无序的整数数组，找出其中最长上升子序列的长度。  
如输入 [ 10, 9, 2, 5, 3, 7, 101, 18 ], 其中最长上升子序列是 [ 2, 3, 7, 101 ], 长度是4。  
注意「⼦序列」和「⼦串」这两个名词的区别，⼦串⼀定是连续的，⽽⼦序列不⼀定是连续的

#### 动态规划解法

动态规划的核⼼设计思想是**数学归纳法**.  

⾸先要定义清楚dp数组的含义，本题可**定义 dp[i] 为以 nums[i] 这个数结尾的最⻓递增⼦序列的⻓度**。

In [74]:
def dpMaxSeries(l):
    n = len(l)
    dp = [1] * n
    
    for i in range(1,n):
        for j in range(i):
            if l[j] < l[i]:
                dp[i] = max(dp[i],1+dp[j])
                
    return max(dp)

In [75]:
l = [10,9,2,5,3,7,101,18]
dpMaxSeries(l)

4

总结⼀下**动态规划的设计流程**：  

**⾸先明确 dp 数组所存数据的含义；  
然后根据 dp 数组的定义，运⽤数学归纳法的思想，假设 dp[0,...,i-1] 都已知，想办法求出 dp[i];  
最后想⼀想问题的base case是什么，以此来初始化 dp 数组，以保证算法正确运⾏.**

### 4. 编辑距离

给定两个字符串 s1、s2，计算将 s1 转换成 s2 的最小操作数。  
你可以对一个字符串进行如下操作：  
1. 插入一个字符；  
2. 删除一个字符；  
3. 替换一个字符。

#### 递归解法

In [77]:
def editDistanceRec(s1,s2):
    if len(s1) == 0: return len(s2)
    if len(s2) == 0: return len(s1)
    
    if s1[-1] == s2[-1]:
        return editDistanceRec(s1[:-1],s2[:-1])
    else:
        return min(
        editDistanceRec(s1,s2[:-1]),
        editDistanceRec(s1[:-1],s2),
        editDistanceRec(s1[:-1],s2[:-1])) + 1

In [78]:
editDistanceRec('horse','ros')

3

In [85]:
comTimeG(editDistanceRec,'hawyacbdaadj','dajkh')

0.02956700325012207

In [89]:
comTimeG(editDistanceRec,'kcwcabwjdaadj','dajkhcenccysgc')

KeyboardInterrupt: 

#### 带备忘录的递归解法

In [95]:
def editDistance2(s1,s2,d):
    if len(s1) == 0: return len(s2)
    if len(s2) == 0: return len(s1)
    if (s1,s2) in d.keys():
        return d[(s1,s2)]
    
    if s1[-1] == s2[-1]:
        d[(s1,s2)] = editDistanceRec(s1[:-1],s2[:-1])
        return d[(s1,s2)]
    else:
        temp = min(
        editDistance2(s1,s2[:-1],d),
        editDistance2(s1[:-1],s2,d),
        editDistance2(s1[:-1],s2[:-1],d)) + 1
        d[(s1,s2)] = temp
        return temp

In [96]:
editDistance2('horse','ros',{})

3

In [97]:
comTimeG(editDistance2,'hawyacbdaadj','dajkh',{})

0.00020694732666015625

In [98]:
comTimeG(editDistance2,'hawyacbhabcjajkcwcabwjdaadj','dajkhcenccysgcehcaiuajdahjakhkaj',{})

KeyboardInterrupt: 

#### 动态规划优化

In [103]:
def editDistanceDP(s1,s2):
    m = len(s1); n = len(s2)
    dp = [[-1]*(n+1)]*(m+1)
    
    # 注意边界条件
    for i in range(m):
        dp[i][0] = i
    for j in range(n):
        dp[0][j] = j
        
    for i in range(1,m+1):
        for j in range(1,n+1):
            if s1[i-1] == s2[j-1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                dp[i][j] = min(
                dp[i-1][j],
                dp[i][j-1],
                dp[i-1][j-1]) + 1
    return dp[m][n]

In [104]:
editDistanceDP('horse','ros')

3

In [105]:
comTimeG(editDistanceDP,'hawyacbhabcjajkcwcabwjdaadj','dajkhcenccysgcehcaiuajdahjakhkaj')

0.0007071495056152344

### 5. 高楼扔鸡蛋

N层楼，K个鸡蛋，假设存在一层楼F，将鸡蛋从这层丢下刚好没碎（高于F的楼层都会碎，低于F的楼层都不会碎）。  
求在最坏情况下，最少需要扔几次鸡蛋，才能确定楼层F？

In [3]:
def superEggDrop(K,N):
    memo = {}
    def dp(K,N):
        if K == 1: return N
        if N == 0: return 0
        if (K,N) in memo.keys():
            return memo[(K,N)]
        res = N
        for i in range(1,N+1):
            res = min(res,
                     max(
                     dp(K,N-i),
                     dp(K-1,i-1)) + 1)
        memo[(K,N)] = res
        return res
    return dp(K,N)

In [4]:
superEggDrop(1,10)

10

In [5]:
superEggDrop(3,10)

4

经分析（见原书），for循环部分，dp关于i是单调的，所以可以用二分查找优化。

In [6]:
def superEggDropB(K,N):
    memo = {}
    def dp(K,N):
        if K == 1: return N
        if N == 0: return 0
        if (K,N) in memo.keys():
            return memo[(K,N)]
        
        res = N
        # 用二分查找替代线性搜索
        lo, hi = 1, N
        while lo <= hi:
            mid = (lo + hi) // 2
            broken = dp(K - 1, mid - 1)
            notBroken = dp(K, N - mid)
            if broken > notBroken:
                hi = mid - 1
                res = min(res,broken + 1)
            else:
                lo = mid + 1
                res = min(res,notBroken + 1)

        memo[(K,N)] = res
        return res
    
    return dp(K,N)

In [7]:
superEggDropB(1,10)

10

In [8]:
superEggDropB(2,10)

4